# Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV
from sklearn.metrics import roc_curve,auc,roc_auc_score
from sklearn.preprocessing import MinMaxScaler

In [2]:
#Read Data_sets
train_data = pd.read_csv('train.csv',encoding = 'ISO-8859-1')
test_data = pd.read_csv('test.csv',encoding='ISO-8859-1')
address_data = pd.read_csv('addresses.csv')
lat_data = pd.read_csv('latlons.csv')

#remove nan in 'compliance' column
train_df = train_data[~train_data.compliance.isnull()]

#appreciate y_label for later
y = train_df.compliance

#combine data sets on train_data
combined = pd.merge(train_df,address_data,how='left',on='ticket_id')
train_df = pd.merge(combined,lat_data,how='left',on='address')

#combine data sets on test_data
combined_test = pd.merge(test_data,address_data,how='left',on='ticket_id')
test_df = pd.merge(combined_test,lat_data,how='left',on='address')

#remove columns cause data leakage and does not exist in test data set
leakage = ['address','violation_street_number','violation_street_name','violation_zip_code','city','state','admin_fee','state_fee']
for i in train_df.columns:
    if i not in test_df.columns:
        leakage.append(i)     
train_df = train_df.drop(train_df[leakage],axis=1)
#print(train_df.columns,np.shape(train_df))

leakage = ['address','violation_street_number','violation_street_name','violation_zip_code','city','state','admin_fee','state_fee']
for i in train_df.columns:
    if i not in test_df.columns:
        leakage.append(i)     
test_df = test_df.drop(test_df[leakage],axis=1)
#print(test_df.columns,np.shape(test_df))



/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#data cleaning in zip_code column.

def code_zip(row):        
    if((type(row) == int) and (row>99999)):
        row = str(row)
        row = int(row[:5])
    return row

train_df['zip_code'] = train_df['zip_code'].apply(code_zip)
test_df['zip_code'] = test_df['zip_code'].apply(code_zip)


In [4]:
print(train_df.columns,np.shape(train_df))
print()
print(test_df.columns,np.shape(test_df))
print()
print(test_df.dtypes)

Index(['ticket_id', 'agency_name', 'inspector_name', 'violator_name',
       'mailing_address_str_number', 'mailing_address_str_name', 'zip_code',
       'non_us_str_code', 'country', 'ticket_issued_date', 'hearing_date',
       'violation_code', 'violation_description', 'disposition', 'fine_amount',
       'late_fee', 'discount_amount', 'clean_up_cost', 'judgment_amount',
       'grafitti_status', 'lat', 'lon'],
      dtype='object') (159880, 22)

Index(['ticket_id', 'agency_name', 'inspector_name', 'violator_name',
       'mailing_address_str_number', 'mailing_address_str_name', 'zip_code',
       'non_us_str_code', 'country', 'ticket_issued_date', 'hearing_date',
       'violation_code', 'violation_description', 'disposition', 'fine_amount',
       'late_fee', 'discount_amount', 'clean_up_cost', 'judgment_amount',
       'grafitti_status', 'lat', 'lon'],
      dtype='object') (61001, 22)

ticket_id                       int64
agency_name                    object
inspector_name     

In [4]:
#combined train and test data sets
temp_df = train_df.append(test_df)
#print(np.shape(temp_df),'159880+61001=',159880+61001)

#replace NaN value with -999
temp_df.fillna(-999,inplace=True)

#sort only object columns in order to use for the conversion to categorical data type.
object_type = []
for i in range(len(temp_df.dtypes)):
    if temp_df.dtypes[i] not in ['float64','int64']:
        object_type.append(temp_df.dtypes.index[i])

#convert object data type to categorical type.       
for col in object_type:
    ucats_an = set(temp_df[col])
    temp_df[col] = pd.Categorical(temp_df[col],categories=ucats_an).codes


#ucats_an = set(temp_df['grafitti_status'])|{'<unknown>'}
#temp_df['grafitti_status']= pd.Categorical(temp_df['grafitti_status'],categories=ucats_an).fillna('<unknown>').codes


In [5]:
#seperate the dataframe into two pieces: train_ddf and test_ddf
train_ddf = temp_df.iloc[:159880,:].set_index('ticket_id')
test_ddf = temp_df.iloc[159880:,:].set_index('ticket_id')


In [29]:
#train dataset normalization
X_train,X_test,y_train,y_test = train_test_split(train_ddf,y)
scale = MinMaxScaler()
X_scale_train = scale.fit_transform(X_train)
X_scale_test = scale.fit_transform(X_test)
#################################################
#test dataset normalization
XX_scale_test = scale.fit_transform(test_ddf)

In [8]:
#modeling
#clf = GradientBoostingClassifier()
#grid_rate = {'learning_rate':[0.25,0.27,0.28,0.29,0.3,0.31],'n_estimators':[300,330,360,400],'max_depth':[3,4,5],'max_features':[None,1,2]}
#grid_cv = GridSearchCV(clf,param_grid=grid_rate,scoring='roc_auc',n_jobs=-1,cv=5)
#grid_fit = grid_cv.fit(X_scale_train,y_train)

In [26]:
y_predicted = GradientBoostingClassifier(n_estimators=400,learning_rate=0.25,max_depth=4,max_features=None).fit(X_scale_train,y_train).predict(X_scale_test)
rs = roc_auc_score(y_test,y_predicted)
rs

KeyboardInterrupt: 

In [27]:
rs

0.57881026305256555

In [24]:
print('compliance:',np.shape(y_train))
print('train_df:',np.shape(train_df))
print('y_predicted:',np.shape(y_predicted))
print('X_train:',np.shape(X_train))
print('X_scale_train:',np.shape(X_scale_train))
print('X_scale_test:',np.shape(X_scale_test))
print('y_test:',np.shape(y_test))


compliance: (119910,)
train_df: (159880, 22)
y_predicted: (39970,)
X_train: (119910, 21)
X_scale_train: (119910, 21)
X_scale_test: (39970, 21)
y_test: (39970,)


In [10]:
print(grid_fit.best_params_,grid_fit.best_score_)

{'learning_rate': 0.25, 'max_depth': 4, 'max_features': None, 'n_estimators': 400} 0.821840264927


In [ ]:
for i in range(len(train_df['zip_code'])):
    if type(train_df['zip_code'][i]) not in [int,'int64']:
        train_df['zip_code'][i] = train_df['zip_code'][i][:5]
    else:
        train_df['zip_code'][i] = str(train_df['zip_code'][i])
        train_df['zip_code'][i] = int(train_df['zip_code'][i][:5])